# Target image analysis

This notebook processes image data from the SNS target imaging system (TIS).

In [ ]:
import sys
from datetime import datetime
import collections
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
import proplot as pplt
import seaborn as sns
from scipy import ndimage

sys.path.append('/Users/46h/Research/')
from accphys.tools import utils

In [ ]:
plt.rcParams['grid.alpha'] = 0.05
plt.rcParams['axes.grid'] = False
plt.rcParams['figure.facecolor'] = 'white'

## Preliminaries 

In [ ]:
pixel_width = 1.0 / 1.77

In [ ]:
TFile = collections.namedtuple('TFile', ['filename', 'timestamp'])

In [ ]:
class Image:
    def __init__(self, Z, pixel_width=1):
        self.Z = Z
        self.Zf = None
        self.n_rows, self.n_cols = Z.shape
        self.xx = np.array(list(range(self.n_cols))).astype(float)
        self.yy = np.array(list(range(self.n_rows))).astype(float)
        self.pixel_width = pixel_width
        self.width = abs(self.xx[-1] - self.xx[0])
        self.height = abs(self.yy[-1] - self.yy[0])
        self.set_pixel_width(pixel_width)
        self.X, self.Y = np.meshgrid(self.xx, self.yy)
        
    def set_pixel_width(self, pixel_width):
        self.xx *= pixel_width
        self.yy *= pixel_width
        
    def filter(self, sigma, **kws):
        Zf = ndimage.gaussian_filter(self.Z, sigma=2*[sigma], order=0, **kws)
        return Zf

In [ ]:
utils.delete_files_not_folders('_output/')

## Load files

In [ ]:
folder = '_saved/2021-09-07/TBT_production_0.5ms/data/'

In [ ]:
tfiles = []
for filename in utils.list_files(folder):
    datetime_str = filename.split('image_')[-1].split('.dat')[0]
    date_str, time_str = datetime_str.split('_')
    year, month, day = [int(s) for s in date_str.split('.')]
    hour, minute, second = [int(s) for s in time_str.split('.')]
    tfiles.append(TFile(filename, datetime(year, month, day, hour, minute, second)))
tfiles = sorted(tfiles, key=lambda tfile: tfile.timestamp)

In [ ]:
images = []
for tfile in tfiles:
    Z = np.loadtxt(tfile.filename).reshape(200, 400)
    Z = np.vstack([np.zeros((100, 400)), Z, np.zeros((100, 400))]) # make square image
    images.append(Image(Z, pixel_width=pixel_width))

## Visualization 

In [ ]:
n_turns_list = list(range(50, 550, 50))

In [ ]:
def add_joint_grid(fig, gridspec, row, col):
    ax_joint = fig.add_subplot(gridspec[row, col])
    ax_marg_x = fig.add_subplot(gridspec[row - 1, col])
    ax_marg_y = fig.add_subplot(gridspec[row, col + 1])
    for ax in [ax_marg_x, ax_marg_y]:
        ax.set_xticks([])
        ax.set_yticks([])
        for side in ['top', 'bottom', 'left', 'right']:
            ax.spines[side].set_visible(False)
    for side in ['top', 'right']:
        ax_joint.spines[side].set_visible(False)
    return ax_joint, ax_marg_x, ax_marg_y

In [ ]:
zoom = 1.25
sigma = 1.75 # for Gaussian filter

In [ ]:
for image, n_turns in zip(images, n_turns_list):
    fig = plt.figure(figsize=(9, 4))
    h = 1.5
    gridspec = fig.add_gridspec(2, 5, width_ratios=(7, h, 2.5, 7, h), height_ratios=(h, 7),
                                left=0.1, right=0.9, bottom=0.1, top=0.9,
                                wspace=0, hspace=0)
    ax1, ax1_marg_x, ax1_marg_y = add_joint_grid(fig, gridspec, 1, 0)
    ax2, ax2_marg_x, ax2_marg_y = add_joint_grid(fig, gridspec, 1, 3)

    joint_kws = dict(cmap='dusk_r')
    marginal_kws = dict(color='black')
    
    X, Y, Z = image.X, image.Y, image.Z
    Zf = image.filter(sigma)
    ax1.pcolormesh(X, Y, Z, cmap='dusk_r', shading='auto')
    ax2.pcolormesh(X, Y, Zf, cmap='dusk_r', shading='auto')
    
    ax1_marg_x.step(image.xx, np.sum(Z, axis=0), **marginal_kws)
    ax1_marg_y.step(np.sum(Z, axis=1), image.yy, **marginal_kws)
    ax2_marg_x.step(image.xx, np.sum(Zf, axis=0), **marginal_kws)
    ax2_marg_y.step(np.sum(Zf, axis=1), image.yy, **marginal_kws)
    
    # Zoom in/out
    new_width = image.width / zoom
    delta_x = 0.5 * (image.width - new_width)
    xmin, xmax = ax1.get_xlim()
    xlim = (xmin + delta_x, xmax - delta_x)
    for ax in [ax1, ax2]:
        ax.set_xlim(xlim)
        ax.set_ylim(xlim)
        
    for ax in [ax1_marg_x, ax2_marg_x]:
        ax.set_xlim(xlim)
    for ax in [ax1_marg_y, ax2_marg_y]:
        ax.set_ylim(xlim)
    for ax in [ax1, ax2]:
        ax.set_xlabel('x [mm]')
        ax.set_ylabel('y [mm]')
        ax.set_aspect(1)
        
    ax2.annotate(r'Gauss filter ($\sigma = {}$)'.format(sigma), xy=(0.01, 0.95), xycoords='axes fraction', color='white', fontsize='small')
    ax1_marg_x.set_title('n_inj_turns = {}'.format(n_turns))
    plt.savefig('_output/target_image_ninjturns={}.png'.format(n_turns), facecolor='white', dpi=300)
    plt.show()